In [1]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import itertools

Using TensorFlow backend.
/home/arslan/.conda/envs/traffic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/arslan/.conda/envs/traffic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/arslan/.conda/envs/traffic/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/arslan/.conda/envs/t

In [2]:
# all images will be converted to this size
ROWS = 300
COLS = 225
CHANNELS = 3

train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('indoor-scenes/train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('indoor-scenes/test', target_size=(ROWS, COLS), class_mode='categorical')

train_generator.reset()
test_generator.reset()

Found 15408 images belonging to 67 classes.
Found 1994 images belonging to 67 classes.


In [12]:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(ROWS, COLS, CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(67))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 223, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 298, 223, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 296, 221, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 296, 221, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 55, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 53, 64)        36928    

In [13]:
tensorboard = TensorBoard(log_dir='./logs_20/custom_20')
model.fit_generator(train_generator, steps_per_epoch=512, epochs=25, callbacks=[tensorboard], verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










Epoch 1/25
 - 479s - loss: 3.9626 - acc: 0.0583

Epoch 2/25
 - 450s - loss: 3.7032 - acc: 0.1031
Epoch 3/25
 - 452s - loss: 3.4852 - acc: 0.1304
Epoch 4/25
 - 455s - loss: 3.3765 - acc: 0.1513
Epoch 5/25
 - 460s - loss: 3.2546 - acc: 0.1702
Epoch 6/25
 - 458s - loss: 3.1620 - acc: 0.1951
Epoch 7/25
 - 460s - loss: 3.0823 - acc: 0.2091
Epoch 8/25
 - 463s - loss: 2.9764 - acc: 0.2338
Epoch 9/25
 - 479s - loss: 2.8972 - acc: 0.2507
Epoch 10/25
 - 496s - loss: 2.8293 - acc: 0.2594
Epoch 11/25
 - 488s - loss: 2.7756 - acc: 0.2767
Epoch 12/25
 - 490s - loss: 2.7120 - acc: 0.2827
Epoch 13/25
 - 499s - loss: 2.6690 - acc: 0.2918
Epoch 14/25
 - 495s - loss: 2.6245 - acc: 0.3117
Epoch 15/25
 - 491s - loss: 2.5741 - acc: 0.3134
Epoch 16/25
 - 490s - loss: 2.5288 - acc: 0.3278
Epoch 17/25
 - 495s - loss: 2.4903 - acc: 0.3315
Epoch 18/25
 - 492s - loss: 2.4408 - acc: 0.3412
Epoch 19/25
 - 495s - 

In [0]:
#import json
# Save JSON config to disk
json_config = model.to_json()
with open('trained_config.json', 'w') as json_file:
    json_file.write(json_config)
# Save weights to disk
model.save_weights('trained_weights.h5')

In [15]:
print(model.evaluate_generator(test_generator, steps=1000))

[2.7757332989804895, 0.34506221459425984]
